In [1]:
require 'csv'
require 'distribution'

true

In [2]:
require './SeedStock_object.rb'

true

In [8]:
require './Gene_object.rb'

false

In [4]:
require './HybridCross_object.rb'

true

In [26]:
# Create an instance of the StockDatabase
#seed_db = SeedStock.new

# Load Seed Stock data from a file
seed_db= SeedStock.load_from_file('StockDatabaseDataFiles/seed_stock_data.tsv')

# Simulate planting 7 grams of seeds and update the gene bank information
seed_db..planting_simulation(7)

# Specify the path to the output TSV file
output_file = 'new_stock_file.tsv'

# Write the updated data to the output file using the StockDatabase
seed_db.class.write_database(output_file)

puts "Data saved to #{output_file}"


Data saved to new_stock_file.tsv


In [23]:
# Create an array to hold instances of SeedStock
seed_stocks = []

# Read the data from the input file
CSV.foreach('StockDatabaseDataFiles/seed_stock_data.tsv', col_sep: "\t", headers: true) do |row|
  # Create a SeedStock object for each row and store it in the array
  seed_stock = SeedStock.new(row['Seed_Stock'], row['Mutant_Gene_ID'], row['Last_Planted'], row['Storage'], row['Grams_Remaining'])
  seed_stocks << seed_stock
end

# Simulate planting 7 grams of seeds and update the gene bank information
seed_stocks.each { |seed_stock| seed_stock.planting_simulation(7) }

# Specify the path to the output TSV file
output_file = 'new_stock_file.tsv'

# Write the updated data to the output file
CSV.open(output_file, 'w', col_sep: "\t") do |csv|
  # Write the header row
  csv << ['Seed_Stock', 'Mutant_Gene_ID', 'Last_Planted', 'Storage', 'Grams_Remaining']
  # Write the data for each SeedStock object to the file
  seed_stocks.each { |seed_stock| csv << seed_stock.to_csv }
end

puts "Data saved to #{output_file}"

Data saved to new_stock_file.tsv


In [12]:
# Read data from seed_stock.tsv and link with gene names
seed_stock_data = CSV.read('StockDatabaseDataFiles/seed_stock_data.tsv', col_sep: "\t", headers: true)
seed_stocks = seed_stock_data.map do |row|
  gene_id = row['Mutant_Gene_ID']
  seed_stock = SeedStock.new(row['Seed_Stock'], gene_id, row['Last_Planted'], row['Storage'], row['Grams_Remaining'])
  seed_stock
end

# Read data from cross_data.tsv
cross_data = CSV.read('StockDatabaseDataFiles/cross_data.tsv', col_sep: "\t", headers: true)

# Create an array to store instances of the HybridCross class
hybrid_crosses = []

# Get the gene_id to gene_name mapping using the class method
gene_id_to_gene_name = Gene.build_gene_id_to_gene_name_mapping

# Iterate through the CSV data and create instances of HybridCross
cross_data.each do |row|
  hybrid_cross = HybridCross.new(
    row['Parent1'],
    row['Parent2'],
    row['F2_Wild'],
    row['F2_P1'],
    row['F2_P2'],
    row['F2_P1P2']
  )
  hybrid_cross.link_to_seed_stocks_and_genes_names(seed_stocks, gene_id_to_gene_name)
  chi_squared = hybrid_cross.chi_squared_test
  hybrid_cross.chi_squared = chi_squared
  hybrid_crosses << hybrid_cross
end

# Link gene names to hybrid crosses
statistically_significant_genes = hybrid_crosses.select { |gene| gene.statistically_significant }

puts "Recording:"
statistically_significant_genes.each do |gene|
  parent1_gene_name = gene.gene_name_parent1
  parent2_gene_name = gene.gene_name_parent2
  chi_squared = gene.chi_squared
  puts "#{parent1_gene_name} is linked to #{parent2_gene_name} with chi-squared score #{format('%.7f', chi_squared)}"
end

puts
puts "Final Report:"

statistically_significant_genes.each do |gene|
  parent1_gene_name = gene.gene_name_parent1
  parent2_gene_name = gene.gene_name_parent2
  puts "#{parent1_gene_name} is linked to #{parent2_gene_name}"
  puts "#{parent2_gene_name} is linked to #{parent1_gene_name}"
end

NameError: uninitialized constant Gene